In [2]:
import scipy.io
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import os
import re
from pathlib import Path
import datetime
import math
import swifter
import math
from scipy import stats
import itertools
from collections import Counter
from statannotations.Annotator import Annotator
import pingouin as pg
import matplotlib.patches as mpatches
import scikit_posthocs as sp
import statsmodels.formula.api as smf
import statsmodels.api as sm
from docx import Document
import gcmi
from openpyxl.styles import PatternFill
# pd.set_option('display.max_rows', None)
plt.rc('xtick',labelsize=12)
plt.rc('ytick',labelsize=12)

/home/morteza/anaconda3/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Functions

In [16]:
def load_data(df):
    fpath=df.path
    mat_data = scipy.io.loadmat(fpath)
    mdata = mat_data['Otot']  # variable in mat file 
    # covnert connections to ROIs names
    group=df.group
    convert_func = np.vectorize(lambda x: groups_ROIs[group].get(x, ''))
    
    # extracting order information
    dic_data={}
    for order in range(3,6):
        for name in mdata.dtype.names:
            column_data=mdata[name][0]
            if name.startswith("index_var"):
                if column_data[order-1].size!=0:
                    dic_data[name + '_' + str(order)]=convert_func(column_data[order-1])
                        
                else:
                    dic_data[name + '_' + str(order)]=np.nan
            else:
                if column_data[order-1].size!=0:
                    dic_data[name + '_' + str(order)]=column_data[order-1]
                        
                else:
                    dic_data[name + '_' + str(order)]=np.nan
    return pd.Series(dic_data)

In [17]:
def preprocess_meta_data(df):
    timepoints=['BI', '4D','7D','10D','12D', '14D', '15D', '16D', '25D']
    epochs=['REM', 'SN', 'VC', 'SLEEP','RUN']
    splitted_path=df.path.split('/')
    
    name_group = [s for s in splitted_path if "Group" in s][0]
    group = int(list(filter(str.isdigit, name_group))[-1]) 
    
#     covnert rats name to their sham or non sham type
    convert_func = np.vectorize(lambda x: type_rat.get(x, ''))
#     remove spaces and convert RAT to R in each name
    name = name_group.split('_')[0].replace(" ", "").replace("AT", "")
    if name =='R12':
        name ='R012'
    df['rat']=name
    df['group']=group
    df['timepoint']=[s for s in splitted_path if s in timepoints][0]
    
#     df['epoch']=[item for item in epochs for s in splitted_path if item.upper() in s.upper()][0]
#     if item.upper() in s.upper()
    df['epoch']=splitted_path[-2]
    df['rat type']=str(convert_func(name))
    return df

In [18]:
def load_data_raw(df):
    dict_data={}
    fpath=df.path
    mat_data = scipy.io.loadmat(fpath)
    try:
        mdata = mat_data['Matrix_2D'] # variable in mat file
        max_timepoint= mat_data['Matrix_2D'].shape[0]
    except:
        mdata = mat_data['Averaged_Samples'] # variable in mat file
        max_timepoint= mat_data['Averaged_Samples'].shape[0]

    return pd.Series({'data':mdata,'data length':max_timepoint})

In [19]:
def rename_type(_type):
    if _type=='red':
        _type='redundant'
    elif _type=='syn':
        _type='synergistic'
    return _type

In [20]:
def sort_date(order,timepoints):
    timepoints=sorted(timepoints, key=lambda x: order.index(x))
    return timepoints

In [21]:
def preprocess_meta_data_raw(df):
    timepoints=['BI', '4D','7D','10D','12D', '14D', '15D', '16D', '25D']
    epochs=['REM', 'SN', 'VC', 'SLEEP','RUN']
    splitted_path=df.path.split('/')
    name_group = [s for s in splitted_path if "Group" in s][0]
    group = int(list(filter(str.isdigit, name_group))[-1]) 
    
    # covnert rats name to their sham or non sham type
    convert_func = np.vectorize(lambda x: type_rat.get(x, ''))
    # remove spaces and convert RAT to R in each name
    name = name_group.split('_')[0].replace(" ", "").replace("AT", "")
    if name =='R12':
        name ='R012'
    df['rat']=name
    df['group']=group
    df['timepoint']=[s for s in splitted_path if s in timepoints][0]
    df['epoch']=splitted_path[-2]                                                 
    df['rat type']=str(convert_func(name))
    if 'Matrix_2D.mat' in splitted_path[-1]:
        df['file_type']= 'mat2d'
    else:
         df['file_type']= splitted_path[-1]
    return df

In [22]:
def extract_id_epoch(epoch_name):
    p = '[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'
    if re.search(p, epoch_name) is not None:
        for catch in re.finditer(p, epoch_name):
            return int(catch[0]) # catch is a match object

In [23]:
def num_interactions(array):
    condition=pd.isna(array)
    if type(condition)==bool and condition:
        return 0
    elif condition.any():
        return 0
    else:
        return array.shape[0]

In [24]:
def plot_text(text,title,ax=None):
    if ax==None:
        fig, ax = plt.subplots()
    # Remove the axis and tick labels
    ax.axis('off')

    # Add the text "There is no data" to the center of the plot
    ax.text(0.5, 0.5, text, ha='center', va='center', fontsize=18)

    # set the title of the figure
    plt.suptitle(title)
    return ax
    

In [25]:
def remove_SE_rats(df,time):
    df_nse=df[(df['rat type']=='NSE')]
    df_sham=df[(df['rat type']=='Sham')]
    df_analyse=pd.concat([df_nse,df_sham])

    return df_analyse

In [26]:
def remove_sham_rats(df,time):
    df_se=df[(df['rat type']=='SE')]
    df_sham_se=df[(df['rat type']=='Sham/SE')]
    df_analyse=pd.concat([df_se,df_sham_se])
    return df_analyse

In [28]:
def timepoint_to_number(column):
    column=column.replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25})
    return column

In [29]:
def rename_epoch(epoch_name):
    if epoch_name=='VC':
        return 'Awake Immobility'
    elif epoch_name=='SN':
        return 'Sniffing'
    elif epoch_name=='SLEEP':
        return 'Sleep'
    elif epoch_name=='REM':
        return 'REM Sleep'
    elif epoch_name=='RUN':
        return 'Running'

In [30]:
palette_epoch = {'Awake Immobility':'#E24A33',
                'Sniffing':'#358ABD',
                'Sleep':'green',
                'REM Sleep':'orange',
                'Running':'darkviolet'}

In [31]:
palette_time = {"BI":"#e60049",
                "4D":"#0bb4ff", 
                "7D":"#50e991",
                "10D":"#e6d800",
                "12D":"#9b19f5",
                "14D":"#ffa300",
                "15D":"#dc0ab4",
                "16D":"#b3d4ff",
                "25D":"#00bfa0"}


In [32]:
epochs=['SN', 'VC', 'SLEEP']

In [33]:
palette_time = {"BI":"#e60049",
                "4D":"#0bb4ff", 
                "7D":"#50e991",
                "10D":"#e6d800",
                "12D":"#9b19f5",
                "14D":"#ffa300",
                "15D":"#dc0ab4",
                "16D":"#b3d4ff",
                "25D":"#00bfa0"}


# Constants

In [34]:
rats_names=['R012', 'R304', 'R305', 'R391', 'R303', 'R319', 'R347', 'R358']
DIR_path=Path(os.getcwd()).parent
SAVE_PATH=DIR_path / 'figures' / datetime.date.today().strftime('%Y_%m_%d')

rat_analyis='R322'

SAVE_PATH_FIG=DIR_path /'results' / datetime.date.today().strftime('%Y_%m_%d') /'figures'
SAVE_PATH_ANALYSIS=DIR_path /'results' / datetime.date.today().strftime('%Y_%m_%d') /'analysis'

# analysis_group='group' # either group or rat
# group_condition=8
# SAVE_PATH=SAVE_PATH 
timepoints=['BI', '4D','7D','10D','12D', '14D', '15D', '16D', '25D']
all_timepoints=['BI', '4D','7D','10D','12D', '14D', '15D', '16D', '25D']

In [35]:
for i in [3,5]:
    if not os.path.exists(SAVE_PATH_FIG/ f'group {i}'/'occurrence'):
        os.makedirs(SAVE_PATH_FIG/ f'group {i}'/'occurrence')

In [36]:
for i in [3,5]:
    if not os.path.exists(SAVE_PATH_ANALYSIS/ f'group {i}'/'occurrence'):
        os.makedirs(SAVE_PATH_ANALYSIS/ f'group {i}'/'occurrence')

In [37]:
groups_ROIs={}
groups_ROIs[1]={
    1:'MS',
    2:'Thal',
    3:'SuM',
    4:'dHPC',
    5:'Subi',
    6:'EC',
    7:'PCC'
   }

groups_ROIs[2]={
    1:'MS',
    2:'dHPC',
    3:'Subi',
    4:'vHPC',
    5:'EC'
}

groups_ROIs[3]={
    1:'MS',
    2:'SuM',
    3:'dHPC',
    4:'EC'
}

groups_ROIs[4]={
    1:'MS',
    2:'dHPC',
    3:'EC'
}

groups_ROIs[5]={
    1:'MS',
    2:'dHPC',
    3:'vHPC',
    4:'Thal'
}

groups_ROIs[6]={
    1:'MS',
    2:'PCC',
    3:'Thal',
    4:'SuM',
    5:'Subi',
    6:'EC'
   }

groups_ROIs[7]={
    1:'MS',
    2:'Thal',
    3:'dHPC',
    4:'vHPC',
    5:'EC',
   }

groups_ROIs[8]={
    1:'MS',
    2:'Thal',
    3:'PCC',
    4:'dHPC',
    5:'vHPC',
    6:'EC'
   }


In [38]:
type_rat={
    'R391':'Sham/SE',
    'R012':'Sham',
    'R304':'Sham',
    'R305':'Sham',
    'R303':'Sham',
    'R358':'Sham/SE',
    'R319':'NSE',
    'R347':'SE',
    'R322':'SE',
    'R326':'SE',
    'R011':'Sham',
    'PILO1':'SE',
    'PILO2':'SE',
    'PILO3':'SE',
    'R322':'SE',
    'R326':'SE'
         }

In [41]:
def get_interactions_per_session(col_interactions):
    interactions = col_interactions.fillna('nan').values
    df_interactions=pd.DataFrame(columns=['session','num_interactions'])
    for session in range(interactions.shape[0]):
        if np.all(np.char.equal(interactions[session], 'nan')):
            df_interactions.loc[len(df_interactions.index)] = [session, 0]
        else:
            df_interactions.loc[len(df_interactions.index)] = [session, interactions[session].shape[0]]
    return df_interactions

# Loading Files

In [42]:
# Specify the parent directory where your files are located
parent_directory_path = f"../data/"

# Use glob to find all files in the parent directory and its subdirectories that end in ".mat"
file_paths = glob.glob(f"{parent_directory_path}/**/results_HOI.mat", recursive=True)

In [43]:
df=pd.DataFrame(file_paths,columns=['path'])

In [44]:
def convert_epochs(epoch_name):
    for epoch in epochs:
        if epoch.upper() in epoch_name.upper():
            return epoch
    return epoch_name

In [46]:
df=df.swifter.apply(preprocess_meta_data, axis=1)

Pandas Apply:   0%|          | 0/5751 [00:00<?, ?it/s]

In [47]:
df['id epoch']=df.epoch.swifter.apply(extract_id_epoch)
df.epoch=df.epoch.swifter.apply(convert_epochs)

Pandas Apply:   0%|          | 0/5751 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/5751 [00:00<?, ?it/s]

# Load higer order interactions

In [48]:
df = pd.concat([df , df.swifter.apply(load_data, axis=1)],axis=1)

Pandas Apply:   0%|          | 0/5751 [00:00<?, ?it/s]

In [49]:
df=df[df.epoch.isin(epochs)]

# Removing outliers

## Loading MI

In [50]:
# Specify the parent directory where your files are located
parent_directory_path = f"../preprocessed_data"

# Use glob to find all files in the parent directory and its subdirectories that end in ".mat"
file_paths = glob.glob(f"{parent_directory_path}/**/*.mat", recursive=True)

In [51]:
df_meta_mi=pd.DataFrame(file_paths,columns=['path'])

In [52]:
df_meta_mi=df_meta_mi.swifter.apply(preprocess_meta_data_raw, axis=1)

Pandas Apply:   0%|          | 0/13921 [00:00<?, ?it/s]

In [53]:
df_meta_mi['id epoch']=df_meta_mi.epoch.swifter.apply(extract_id_epoch)
df_meta_mi.epoch=df_meta_mi.epoch.swifter.apply(convert_epochs)

Pandas Apply:   0%|          | 0/13921 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/13921 [00:00<?, ?it/s]

In [54]:
df_meta_mi

,path,rat,group,timepoint,epoch,rat type,file_type,id epoch
0,../preprocessed_data/GROUP 2/R 304_PREPARED FI...,R304,2,BI,SLEEP,Sham,mat2d,3.0
1,../preprocessed_data/GROUP 2/R 304_PREPARED FI...,R304,2,BI,SLEEP,Sham,ROI1_Averaged.mat,3.0
2,../preprocessed_data/GROUP 2/R 304_PREPARED FI...,R304,2,BI,SLEEP,Sham,ROI2_Averaged.mat,3.0
3,../preprocessed_data/GROUP 2/R 304_PREPARED FI...,R304,2,BI,SLEEP,Sham,ROI3_Averaged.mat,3.0
4,../preprocessed_data/GROUP 2/R 304_PREPARED FI...,R304,2,BI,SLEEP,Sham,ROI4_Averaged.mat,3.0
...,...,...,...,...,...,...,...,...
13916,../preprocessed_data/R326_PREPARED/R322_Group ...,R322,5,7D,VC,SE,mat2d,4.0
13917,../preprocessed_data/R326_PREPARED/R322_Group ...,R322,5,7D,VC,SE,mat2d,5.0
13918,../preprocessed_data/R326_PREPARED/R322_Group ...,R322,5,7D,VC,SE,mat2d,6.0
13919,../preprocessed_data/R326_PREPARED/R322_Group ...,R322,5,7D,VC,SE,mat2d,7.0


In [55]:
df_meta_mi=df_meta_mi[df_meta_mi.epoch.isin(epochs)]

## Loading MI

In [56]:
df_meta_mi=df_meta_mi[df_meta_mi.file_type=='mat2d']

In [57]:
df_meta_mi = pd.concat([df_meta_mi , df_meta_mi.swifter.apply(load_data_raw, axis=1)],axis=1)

Pandas Apply:   0%|          | 0/3800 [00:00<?, ?it/s]

In [58]:
def num_interactions(array):
    condition=pd.isna(array)
    if type(condition)==bool and condition:
        return 0
    elif condition.any():
        return 0
    else:
        return array.shape[0]

# Concat MI and Interactions

In [59]:
df_total = df.merge(df_meta_mi, on=['group', 'rat','timepoint','epoch','id epoch'],suffixes=('_interaction','_mi'))
timepoints=sort_date(order=all_timepoints,timepoints=df_total.timepoint.unique().tolist())
df_total['timepoint'] = pd.Categorical(
df_total['timepoint'], 
categories=timepoints, 
ordered=True)

In [60]:
df_total=df_total[df_total.group.isin([3,5])]

In [61]:
for order in [3,4]:
    for _type in ['red','syn']:
        df_total[f'num_{_type}_{order}']=df_total[f'index_var_{_type}_{order}'].apply(num_interactions)

In [62]:
for _type in ['red','syn']:
    df_total[f'num_{_type}_total']=df_total[f'num_{_type}_3']+df_total[f'num_{_type}_4']

# Find Q1 and Q3 4th order interactions

In [63]:
df_quantile_time = df_total[df_total['num_red_3']==4]

In [64]:
df_quantile_time = df_quantile_time.groupby('epoch')['data length'].quantile([0.10, 0.80]) \
         .unstack() \
         .rename(columns={0.10:'Q1', 0.80:'Q3'})

In [65]:
df_removed_outliers=pd.DataFrame([])
for epoch in df_quantile_time.index:
    min_data = df_quantile_time[df_quantile_time.index==epoch]['Q1'].values[0]
    max_data = df_quantile_time[df_quantile_time.index==epoch]['Q3'].values[0]
    df_epoch = df_total[(df_total.epoch==epoch)&(df_total['data length']>min_data) &(df_total['data length']<max_data)]
    df_removed_outliers = pd.concat([df_removed_outliers,df_epoch])

In [66]:
df = df_removed_outliers
df['rat type']=df['rat type_interaction']

In [67]:
df=remove_sham_rats(df,'ALL')

In [68]:
df['timepoint'] =(df['timepoint']).astype(str)

# Funtions stats

In [79]:

# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

In [80]:
def calculate_effect_size(df_data,df_posthoc,formula,column):
    effect_sizes=np.empty_like(df_posthoc)
    effect_sizes[:] = np.nan
    np.fill_diagonal(effect_sizes, 0)
    timepoints=np.sort(df_data.timepoint.unique())
    if formula=='cohen':
        for i,t1 in enumerate(timepoints):
            for j,t2 in enumerate(timepoints):
                if t1==t2:
                    continue
                effect_sizes[i,j]=cohend(df_data[df_data.timepoint==t1][column],df_data[df_data.timepoint==t2][column])
        effect_sizes=np.abs(effect_sizes)
    elif formula=='r':
        for counter, (group_pair, p_value_posthoc) in enumerate(df_posthoc.iterrows()):
            z_value = stats.norm.ppf(1 - p_value_posthoc / 2)  # Two-tailed test
            ef_row = z_value / np.sqrt(df_data.shape[0])
            ef_row[np.isinf(ef_row)] = 0 # replace inf with 0
            effect_sizes[counter]=np.abs(ef_row)
    # Round the effect sizes to 3 decimal places
    effect_sizes = effect_sizes.round(3)
    return effect_sizes
    

In [81]:

def rm_test(df_test,col='num_interactions'):
    result_stat = stats.kruskal(*list(df_test.groupby('timepoint').agg(list)[f'{col}'].values))
    H=result_stat[0]
    pval=result_stat[1]
    n=len(list(df_test[f'{col}'].values))
    k=len(df_test.timepoint.unique())
    print(H,n,k)
    esq = (H -k +1)/(n-k)
    print(esq)
    return pval,esq

In [82]:
# Function to randomly keep n rows for each category
def keep_n_rows_per_category(df, category_column, n):
    result_df=df.groupby(category_column).apply(lambda x: x.sample(min(len(x), n))).reset_index(drop=True)
    result_df['order'] = result_df.groupby(category_column).cumcount() + 1
    return result_df

In [83]:
condition='all'

In [84]:
df=remove_sham_rats(df,'ALL TIMEPOINTS')

# Stats Mutual Info

In [ ]:
def calculate_mutal_info(df_analysis):
    n_region=df_analysis['data'].iloc[0].shape[1]
    mutal_info=np.zeros((df_analysis.shape[0],n_region,n_region))
    for n in range(df_analysis.shape[0]):
        for i in range(n_region):
            for j in range(n_region):
                if i==j:
                    continue
                mutal_info[n,i,j]=gcmi.gcmi_cc(df_analysis['data'].values[n][:,i],df_analysis['data'].values[n][:,j])
    return mutal_info

In [ ]:
def get_mi_epoch(df,group,epoch):
    df_group=df[df.group==group]

    df_epoch=df_group[df_group.epoch==epoch]
    df_mi=pd.DataFrame()
    for timepoint in df_epoch.timepoint.unique():
        df_analysis=df_epoch[(df_epoch.timepoint==timepoint)]
        #calculate mutual inofrmation for all datas
        mutual_info=calculate_mutal_info(df_analysis)
        # reshape it from n_samples,n_regions,n_regions to n_samples,n_regions*n_regions
        mutual_info=mutual_info.reshape(mutual_info.shape[0],-1)
        # create columns based on the group
        regions_lables=list(groups_ROIs[group].values())
        cols =list(itertools.product(regions_lables,regions_lables))
        # create a dataframe from the results
        df_mi_single_timepoint=pd.DataFrame(mutual_info,columns=cols)
        # select only pairwise interaction and remove redunent one
        df_mi_single_timepoint = df_mi_single_timepoint[list(itertools.combinations(regions_lables, 2))]
        # add timepoint columns
        df_mi_single_timepoint['timepoint']=timepoint

        # concat with all timepoints
        df_mi=pd.concat([df_mi,df_mi_single_timepoint],ignore_index=True)
    return df_mi

In [ ]:
for group in [3,5]:
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_group_{group}_mutual_info_regional_cohen.xlsx',engine='openpyxl') as writer:
        df_group = df[(df.group==group)]
        for epoch in df_group.epoch.unique():
            mutual_info_all={}
            df_epoch=df_group[df_group.epoch==epoch]
            #remove sham
            df_epoch=remove_sham_rats(df_epoch,'ALL TIMEPOINTS')
            for timepoint in df_epoch.timepoint.unique():
                df_analysis=df_epoch[(df_epoch.timepoint==timepoint)]
                mutal_info=calculate_mutal_info(df_analysis)
                mutual_info_all[timepoint]=mutal_info
    
            # Create tags-timepoint array for np array
            num_rows_per_array = [arr.shape[0] for arr in mutual_info_all.values()]
            tags = np.concatenate([np.full(shape, key) for key, shape in zip(mutual_info_all.keys(), num_rows_per_array)])
            #concanate array from list of (n,m,m) --> (n*timepoints,m,m)
            mutual_info_all=np.concatenate(list(mutual_info_all.values()),axis=0)
    
            ########################stats####################################
            
            for i in range(mutual_info_all.shape[1]):
                df_data_reapeated_measure=pd.DataFrame()
                df_results_all=pd.DataFrame()
                for timepoint in df_epoch.timepoint.unique():
                    #  convert np array of timepoints to dataframe of timepoints
                    if df_data_reapeated_measure.shape[0]==0:
                        df_data_reapeated_measure=pd.DataFrame(mutual_info_all[tags==timepoint][:,i,:].flatten(),columns=['mutual info'])
                        df_data_reapeated_measure['timepoint']=timepoint
                    else:
                        row=pd.DataFrame(mutual_info_all[tags==timepoint][:,i,:].flatten(),columns=['mutual info'])
                        row['timepoint']=timepoint
                        df_data_reapeated_measure=pd.concat([df_data_reapeated_measure,row])

                # convert timepoint to number
                df_data_reapeated_measure['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                   ,inplace=True)
                # pass the dataframe of timepoints for stats. this contatin mutual infro between ROI i
                if len(df_data_reapeated_measure.timepoint.unique())==1:
                    continue
                p_value,esq=rm_test(df_data_reapeated_measure,'mutual info')
                print(3*'*')
                print(group,epoch,f'{groups_ROIs[group][i+1]}',p_value)
                print(3*'*')
                if p_value<0.05:
                    df_posthoc = sp.posthoc_conover(df_data_reapeated_measure, val_col='mutual info', group_col='timepoint', p_adjust = 'holm')
                    effect_sizes=calculate_effect_size(df_data_reapeated_measure,df_posthoc,'cohen',column='mutual info')
                    # Convert to DataFrame
                    effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                    df_results_all=pd.concat([df_results_all,df_posthoc])
            
                # add pvalue-kw
                # new_row_data={0:p_value,1:esq}
                # new_index = pd.MultiIndex.from_tuples([(order,'KW-pvalue')], names=[ None])
                new_row = pd.DataFrame([p_value,esq], index=['KW-pvalue','Effect Size'],columns=[0])
                df_results_all=pd.concat([new_row,df_results_all])
                # df_results_all=df_results_all.sort_index()
            
                print(epoch)
                df_results_all.to_excel(writer, sheet_name=f'epoch {epoch} - mutual info {groups_ROIs[group][i+1]}')
                effect_sizes_df.to_excel(writer, sheet_name=f'ES - epoch {epoch} - mutual info {groups_ROIs[group][i+1]}')
                
                # formating values in exel file
                # Get the xlsxwriter workbook and worksheet objects
                workbook = writer.book
                worksheet = writer.sheets[f'epoch {epoch} - mutual info {groups_ROIs[group][i+1]}']
                # Define a cell format with red fill for cells greater than 5
                red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')
                
                # Iterate through the DataFrame and apply formatting to cells greater than 5
                for row in range(2, df_results_all.shape[0] + 73):
                    for col in range(2, df_results_all.shape[1] + 3):
                        cell_value = worksheet.cell(row=row, column=col).value
                        if cell_value is not None and cell_value != '' and cell_value < 0.05:
                            worksheet.cell(row=row, column=col).fill = red_fill
                # mutual_info_all

In [ ]:
for group in [3,5]:
    df_group = df[(df.group==group)]
    for epoch in df_group.epoch.unique():
        mutual_info_all={}
        df_epoch=df_group[df_group.epoch==epoch]
        #remove sham
        df_epoch=remove_sham_rats(df_epoch,'ALL TIMEPOINTS')
        for timepoint in df_epoch.timepoint.unique():
            df_analysis=df_epoch[(df_epoch.timepoint==timepoint)]
            mutal_info=calculate_mutal_info(df_analysis)
            mutual_info_all[timepoint]=mutal_info

        # Create tags-timepoint array for np array
        num_rows_per_array = [arr.shape[0] for arr in mutual_info_all.values()]
        tags = np.concatenate([np.full(shape, key) for key, shape in zip(mutual_info_all.keys(), num_rows_per_array)])
        #concanate array from list of (n,m,m) --> (n*timepoints,m,m)
        mutual_info_all=np.concatenate(list(mutual_info_all.values()),axis=0)

        ########################stats####################################
        
        for i in range(mutual_info_all.shape[1]):
            df_data_reapeated_measure=pd.DataFrame()
            df_results_all=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique():
                #  convert np array of timepoints to dataframe of timepoints
                if df_data_reapeated_measure.shape[0]==0:
                    df_data_reapeated_measure=pd.DataFrame(mutual_info_all[tags==timepoint][:,i,:].flatten(),columns=['mutual info'])
                    df_data_reapeated_measure['timepoint']=timepoint
                else:
                    row=pd.DataFrame(mutual_info_all[tags==timepoint][:,i,:].flatten(),columns=['mutual info'])
                    row['timepoint']=timepoint
                    df_data_reapeated_measure=pd.concat([df_data_reapeated_measure,row])


In [ ]:
for group in [3,5]:
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_group_{group}_mutual_info_cohen.xlsx',engine='openpyxl') as writer:
        df_group = df[(df.group==group)]
        for epoch in df_group.epoch.unique():
            mutual_info_all={}
            df_epoch=df_group[df_group.epoch==epoch]
            #remove sham
            df_epoch=remove_sham_rats(df_epoch,'ALL TIMEPOINTS')
            for timepoint in df_epoch.timepoint.unique():
                df_analysis=df_epoch[(df_epoch.timepoint==timepoint)]
                mutal_info=calculate_mutal_info(df_analysis)
                mutual_info_all[timepoint]=mutal_info
    
            # Create tags-timepoint array for np array
            num_rows_per_array = [arr.shape[0] for arr in mutual_info_all.values()]
            tags = np.concatenate([np.full(shape, key) for key, shape in zip(mutual_info_all.keys(), num_rows_per_array)])
            #concanate array from list of (n,m,m) --> (n*timepoints,m,m)
            mutual_info_all=np.concatenate(list(mutual_info_all.values()),axis=0)
    
            ########################stats####################################
            
            for i in range(mutual_info_all.shape[1]):
                for j in range(i+1,mutual_info_all.shape[2]):
                    df_data_reapeated_measure=pd.DataFrame()
                    df_results_all=pd.DataFrame()
                    for timepoint in df_epoch.timepoint.unique():
    
                        #  convert np array of timepoints to dataframe of timepoints
                        if df_data_reapeated_measure.shape[0]==0:
                            df_data_reapeated_measure=pd.DataFrame(mutual_info_all[tags==timepoint][:,i,j],columns=['mutual info'])
                            df_data_reapeated_measure['timepoint']=timepoint
                        else:
                            row=pd.DataFrame(mutual_info_all[tags==timepoint][:,i,j],columns=['mutual info'])
                            row['timepoint']=timepoint
                            df_data_reapeated_measure=pd.concat([df_data_reapeated_measure,row])

                    # convert timepoint to number
                    df_data_reapeated_measure['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                       ,inplace=True)
                    if len(df_data_reapeated_measure.timepoint.unique())==1:continue
                    # pass the dataframe of timepoints for stats. this contatin mutual infro between ROI i and j
                    p_value,esq=rm_test(df_data_reapeated_measure,'mutual info')
                    if p_value<0.05:
                        df_posthoc = sp.posthoc_conover(df_data_reapeated_measure, val_col='mutual info', group_col='timepoint', p_adjust = 'holm')
                        effect_sizes=calculate_effect_size(df_data_reapeated_measure,df_posthoc,'cohen',column='mutual info')
                        # Convert to DataFrame
                        effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                        df_results_all=pd.concat([df_results_all,df_posthoc])
                
                    # add pvalue-kw
                    # new_row_data={0:p_value,1:esq}
                    # new_index = pd.MultiIndex.from_tuples([(order,'KW-pvalue')], names=[ None])
                    new_row = pd.DataFrame([p_value,esq], index=['KW-pvalue','Effect Size'],columns=[0])
                    df_results_all=pd.concat([new_row,df_results_all])
                    # df_results_all=df_results_all.sort_index()
                
                    print(epoch)
                    df_results_all.to_excel(writer, sheet_name=f'epoch {epoch} - mutual info {groups_ROIs[group][i+1]} and {groups_ROIs[group][j+1]}')
                    effect_sizes_df.to_excel(writer, sheet_name=f'ES - epoch {epoch} - mutual info {groups_ROIs[group][i+1]} and {groups_ROIs[group][j+1]}')
                    
                    # formating values in exel file
                    # Get the xlsxwriter workbook and worksheet objects
                    workbook = writer.book
                    worksheet = writer.sheets[f'epoch {epoch} - mutual info {groups_ROIs[group][i+1]} and {groups_ROIs[group][j+1]}']
                    # Define a cell format with red fill for cells greater than 5
                    red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')
                    
                    # Iterate through the DataFrame and apply formatting to cells greater than 5
                    for row in range(2, df_results_all.shape[0] + 73):
                        for col in range(2, df_results_all.shape[1] + 3):
                            cell_value = worksheet.cell(row=row, column=col).value
                            if cell_value is not None and cell_value != '' and cell_value < 0.05:
                                worksheet.cell(row=row, column=col).fill = red_fill
                    # mutual_info_all

In [ ]:
def upper_tri_masking(A):
    m, n, _ = A.shape
    r = np.arange(n)
    mask = r[:, None] < r
    return A[:, mask]

In [ ]:
for group in [3,5]:
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_group_{group}_mutual_info_all_regions_cohen.xlsx',engine='openpyxl') as writer:
        df_group = df[(df.group==group)]
        for epoch in df_group.epoch.unique():
            mutual_info_all={}
            df_epoch=df_group[df_group.epoch==epoch]
            #remove sham
            df_epoch=remove_sham_rats(df_epoch,'ALL TIMEPOINTS')
            for timepoint in df_epoch.timepoint.unique():
                df_analysis=df_epoch[(df_epoch.timepoint==timepoint)]
                mutal_info=calculate_mutal_info(df_analysis)
                mutual_info_all[timepoint]=mutal_info
    
            # Create tags-timepoint array for np array
            num_rows_per_array = [arr.shape[0] for arr in mutual_info_all.values()]
            tags = np.concatenate([np.full(shape, key) for key, shape in zip(mutual_info_all.keys(), num_rows_per_array)])
            #concanate array from list of (n,m,m) --> (n*timepoints,m,m)
            mutual_info_all=np.concatenate(list(mutual_info_all.values()),axis=0)
    
            ########################stats####################################
            

            df_data_reapeated_measure=pd.DataFrame()
            df_results_all=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique():

                #  convert np array of timepoints to dataframe of timepoints
                if df_data_reapeated_measure.shape[0]==0:
                    df_data_reapeated_measure=pd.DataFrame(upper_tri_masking(mutual_info_all[tags==timepoint]).flatten(),columns=['mutual info'])
                    df_data_reapeated_measure['timepoint']=timepoint
                else:
                    row=pd.DataFrame(upper_tri_masking(mutual_info_all[tags==timepoint]).flatten(),columns=['mutual info'])
                    row['timepoint']=timepoint
                    df_data_reapeated_measure=pd.concat([df_data_reapeated_measure,row])

            # convert timepoint to number
            df_data_reapeated_measure['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                               ,inplace=True)
            
            if len(df_data_reapeated_measure.timepoint.unique())==1:continue
            # pass the dataframe of timepoints for stats. this contatin mutual infro between ROI i and j
            p_value,esq=rm_test(df_data_reapeated_measure,'mutual info')
            if p_value<0.05:
                df_posthoc = sp.posthoc_conover(df_data_reapeated_measure, val_col='mutual info', group_col='timepoint', p_adjust = 'holm')
                effect_sizes=calculate_effect_size(df_data_reapeated_measure,df_posthoc,'cohen',column='mutual info')
                # Convert to DataFrame
                effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                df_results_all=pd.concat([df_results_all,df_posthoc])
        
            # add pvalue-kw
            # new_row_data={0:p_value}
            # new_index = pd.MultiIndex.from_tuples([(order,'KW-pvalue')], names=[ None])
            new_row = pd.DataFrame([p_value,esq], index=['KW-pvalue','Effect Size'],columns=[0])
            df_results_all=pd.concat([new_row,df_results_all])
            # df_results_all=df_results_all.sort_index()
        
            print(epoch)
            df_results_all.to_excel(writer, sheet_name=f'epoch {epoch} - mutual info')
            effect_sizes_df.to_excel(writer, sheet_name=f'ES - epoch {epoch} - mutual info')
            
            # formating values in exel file
            # Get the xlsxwriter workbook and worksheet objects
            workbook = writer.book
            worksheet = writer.sheets[f'epoch {epoch} - mutual info']
            # Define a cell format with red fill for cells greater than 5
            red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')
            
            # Iterate through the DataFrame and apply formatting to cells greater than 5
            for row in range(2, df_results_all.shape[0] + 73):
                for col in range(2, df_results_all.shape[1] + 3):
                    cell_value = worksheet.cell(row=row, column=col).value
                    if cell_value is not None and cell_value != '' and cell_value < 0.05:
                        worksheet.cell(row=row, column=col).fill = red_fill
            # mutual_info_all

# O value

In [ ]:
def preprocess_o_values(df,order,_type,region=False):
    df_analysis = df.explode([f'index_var_{_type}_{order}',f'sorted_{_type}_{order}']).reset_index(drop=True)
    df_analysis=df_analysis[df_analysis.group==group]
    df_analysis=df_analysis=remove_sham_rats(df_analysis,'ALL TIMEPOINTS')
    # remove nan rows
    df_analysis=df_analysis[~df_analysis[f'index_var_{_type}_{order}'].isna()]
    
    if region:
        df_analysis[f'sorted_{_type}_{order}']=df_analysis[f'sorted_{_type}_{order}'].apply(lambda x: np.repeat(x,order))
        df_analysis=df_analysis.explode([f'index_var_{_type}_{order}',f'sorted_{_type}_{order}']).reset_index(drop=True)
    else:
        df_analysis=df_analysis.explode([f'sorted_{_type}_{order}']).reset_index(drop=True)
    return df_analysis

In [ ]:
results=pd.DataFrame()
for _type in ['red','syn']:
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_{_type}_ovalues_cohen.xlsx',engine='openpyxl') as writer:
        for group in [3,5]:
            df_group = df[(df.group==group)]
            for epoch in df_group.epoch.unique():
                df_epoch=df_group[df_group.epoch==epoch]
    
                df_results_all_orders=pd.DataFrame([])
                effect_sizes_all=pd.DataFrame([])
                for order in [3,4]:
                    df_analysis=preprocess_o_values(df_epoch,order,_type)
                    if df_analysis.shape[0]==0:continue
                    data=np.hstack(df_analysis[f'sorted_{_type}_{order}'].dropna().values)
        #             min_timepoint =df_order.groupby('timepoint').count().min().min()
        #             df_order = keep_n_rows_per_category(df_order,'timepoint',min_timepoint)
                    
                    if data.shape[0]==0 or df_analysis.timepoint.unique().shape[0]==1: continue
                    df_analysis['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                       ,inplace=True)
    
                    p_value,esq =rm_test(df_analysis,f'sorted_{_type}_{order}')
                    if p_value<0.05:
                        df_posthoc = sp.posthoc_conover(df_analysis, val_col=f'sorted_{_type}_{order}', group_col='timepoint', p_adjust = 'holm')
                        effect_sizes=calculate_effect_size(df_analysis,df_posthoc,'cohen',column=f'sorted_{_type}_{order}')
                        # Convert to DataFrame
                        effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                        effect_sizes_df['order']=order
                        effect_sizes_df=effect_sizes_df.set_index(['order',effect_sizes_df.index])
                        effect_sizes_all=pd.concat([effect_sizes_all,effect_sizes_df])
                        
                        df_posthoc['order']=order
                        df_posthoc=df_posthoc.set_index(['order',df_posthoc.index])
                        df_results_all_orders=pd.concat([df_results_all_orders,df_posthoc])
    
                    # add pvalue-kw
                    
                    new_index = pd.MultiIndex.from_tuples([(order,'KW-pvalue'),(order,'Effect Size')], names=['order', None])
                    new_row = pd.DataFrame([p_value,esq], index=new_index,columns=[0])
                    df_results_all_orders=pd.concat([new_row,df_results_all_orders])
                    df_results_all_orders=df_results_all_orders.sort_index()
    
                print(epoch)
                df_results_all_orders.to_excel(writer, sheet_name=f'group {group} - epoch {epoch}')
                if effect_sizes_all.shape[0]!=0:
                    effect_sizes_all.to_excel(writer, sheet_name=f'ES - group {group} - epoch {epoch}')

                # formating values in exel file
                # Get the xlsxwriter workbook and worksheet objects
                workbook = writer.book
                worksheet = writer.sheets[f'group {group} - epoch {epoch}']
                # Define a cell format with red fill for cells greater than 5
                red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')
    
                # Iterate through the DataFrame and apply formatting to cells greater than 5
                for row in range(2, df_results_all_orders.shape[0] + 73):
                    for col in range(3, df_results_all_orders.shape[1] + 3):
                        cell_value = worksheet.cell(row=row, column=col).value
                        if cell_value is not None and cell_value != '' and cell_value < 0.05:
                            worksheet.cell(row=row, column=col).fill = red_fill

In [ ]:
results=pd.DataFrame()
for _type in ['red','syn']:
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_{_type}_ovalues_interactions_cohen.xlsx',engine='openpyxl') as writer:
        for group in [3,5]:
            df_group = df[(df.group==group)]
            for epoch in df_group.epoch.unique():
                df_epoch=df_group[df_group.epoch==epoch]
    
                df_results_all_orders=pd.DataFrame([])
                effect_sizes_all=pd.DataFrame([])
                for order in [3,4]:
                    df_ovalues=preprocess_o_values(df_epoch,order,_type)
                    if df_ovalues.shape[0]==0:continue
                    data=np.hstack(df_ovalues[f'sorted_{_type}_{order}'].dropna().values)
        #             min_timepoint =df_order.groupby('timepoint').count().min().min()
        #             df_order = keep_n_rows_per_category(df_order,'timepoint',min_timepoint)
                    
                    if data.shape[0]==0 or df_ovalues.timepoint.unique().shape[0]==1: continue

                    df_ovalues['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                   ,inplace=True)

                    df_ovalues[f'index_var_{_type}_{order}']=df_ovalues[f'index_var_{_type}_{order}'].astype(str)
                    interactions = np.unique(df_ovalues[f'index_var_{_type}_{order}'])
                    interactions = np.delete(interactions,np.argwhere(interactions=='nan'))
                    
                    for interact in interactions:
                        df_analysis=df_ovalues[df_ovalues[f'index_var_{_type}_{order}']==interact]
                        if len(df_analysis.timepoint.unique())==1:continue
                        p_value,esq =rm_test(df_analysis,f'sorted_{_type}_{order}')
                        if p_value<0.05:
                            df_posthoc = sp.posthoc_conover(df_analysis, val_col=f'sorted_{_type}_{order}', group_col='timepoint', p_adjust = 'holm')

                            effect_sizes=calculate_effect_size(df_analysis,df_posthoc,'cohen',column=f'sorted_{_type}_{order}')
                        
                            # Convert to DataFrame
                            effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                            effect_sizes_df['order']=order
                            effect_sizes_df['ineteraction']=interact
                            effect_sizes_df=effect_sizes_df.set_index(['order','ineteraction',effect_sizes_df.index])
                            effect_sizes_all=pd.concat([effect_sizes_all,effect_sizes_df])
                        
                            df_posthoc['order']=order
                            df_posthoc['ineteraction']=interact
                            df_posthoc=df_posthoc.set_index(['order','ineteraction',df_posthoc.index])
                            df_results_all_orders=pd.concat([df_results_all_orders,df_posthoc])
        
                        # add pvalue-kw
                        new_index = pd.MultiIndex.from_tuples([(order,interact,'KW-pvalue'),(order,interact,'Effect Size')], names=['order', None,None])
                        new_row = pd.DataFrame([p_value,esq], index=new_index,columns=[0])
                        df_results_all_orders=pd.concat([new_row,df_results_all_orders])
                        df_results_all_orders=df_results_all_orders.sort_index()
        
                    print(epoch)
                    df_results_all_orders.to_excel(writer, sheet_name=f'group {group} - epoch {epoch}')
                    effect_sizes_all.to_excel(writer, sheet_name=f'ES - group {group} - epoch {epoch}')
    
                    # formating values in exel file
                    # Get the xlsxwriter workbook and worksheet objects
                    workbook = writer.book
                    worksheet = writer.sheets[f'group {group} - epoch {epoch}']
                    # Define a cell format with red fill for cells greater than 5
                    red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')
        
                # Iterate through the DataFrame and apply formatting to cells greater than 5
                for row in range(2, df_results_all_orders.shape[0] + 200):
                    for col in range(4, df_results_all_orders.shape[1] + 5):
                        cell_value = worksheet.cell(row=row, column=col).value
                        if cell_value is not None and cell_value != '' and cell_value < 0.05:
                            worksheet.cell(row=row, column=col).fill = red_fill

In [77]:

# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

In [ ]:
results=pd.DataFrame()
for _type in ['red','syn']:
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_{_type}_ovalues_regional_cohen.xlsx',engine='openpyxl') as writer:
        for group in [3,5]:
            df_group = df[(df.group==group)]
            for epoch in df_group.epoch.unique():
                df_epoch=df_group[df_group.epoch==epoch]
    
                df_results_all_orders=pd.DataFrame([])
                effect_sizes_all=pd.DataFrame([])
                for order in [3,4]:
                    df_ovalues=preprocess_o_values(df_epoch,order,_type,True)
                    if df_ovalues.shape[0]==0:continue
                    data=np.hstack(df_ovalues[f'sorted_{_type}_{order}'].dropna().values)
        #             min_timepoint =df_order.groupby('timepoint').count().min().min()
        #             df_order = keep_n_rows_per_category(df_order,'timepoint',min_timepoint)
                    
                    if data.shape[0]==0 or df_ovalues.timepoint.unique().shape[0]==1: continue

                    df_ovalues['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                   ,inplace=True)

                    df_ovalues[f'index_var_{_type}_{order}']=df_ovalues[f'index_var_{_type}_{order}'].astype(str)
                    regions = np.unique(df_ovalues[f'index_var_{_type}_{order}'])
                    regions = np.delete(regions,np.argwhere(regions=='nan'))
                    
                    for region in regions:
                        df_analysis=df_ovalues[df_ovalues[f'index_var_{_type}_{order}']==region]
                        p_value,esq =rm_test(df_analysis,f'sorted_{_type}_{order}')
                        if p_value<0.05:
                            df_posthoc = sp.posthoc_conover(df_analysis, val_col=f'sorted_{_type}_{order}', group_col='timepoint', p_adjust = 'holm')

                            effect_sizes=calculate_effect_size(df_analysis,df_posthoc,'cohen',column=f'sorted_{_type}_{order}')
                            # Convert to DataFrame
                            effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                            effect_sizes_df['order']=order
                            effect_sizes_df['region']=region
                            effect_sizes_df=effect_sizes_df.set_index(['order','region',effect_sizes_df.index])
                            effect_sizes_all=pd.concat([effect_sizes_all,effect_sizes_df])
                        
                            df_posthoc['order']=order
                            df_posthoc['region']=region
                            df_posthoc=df_posthoc.set_index(['order','region',df_posthoc.index])
                            df_results_all_orders=pd.concat([df_results_all_orders,df_posthoc])
        
                        # add pvalue-kw
                        new_index = pd.MultiIndex.from_tuples([(order,region,'KW-pvalue'),(order,region,'Effect Size')], names=['order', None,None])
                        new_row = pd.DataFrame([p_value,esq], index=new_index,columns=[0])
                        df_results_all_orders=pd.concat([new_row,df_results_all_orders])
                        df_results_all_orders=df_results_all_orders.sort_index()
        
                    print(epoch)
                    df_results_all_orders.to_excel(writer, sheet_name=f'group {group} - epoch {epoch}')
                    effect_sizes_all.to_excel(writer, sheet_name=f'ES - group {group} - epoch {epoch}')
    
                    # formating values in exel file
                    # Get the xlsxwriter workbook and worksheet objects
                    workbook = writer.book
                    worksheet = writer.sheets[f'group {group} - epoch {epoch}']
                    # Define a cell format with red fill for cells greater than 5
                    red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')
        
                # Iterate through the DataFrame and apply formatting to cells greater than 5
                for row in range(2, df_results_all_orders.shape[0] + 200):
                    for col in range(4, df_results_all_orders.shape[1] + 5):
                        cell_value = worksheet.cell(row=row, column=col).value
                        if cell_value is not None and cell_value != '' and cell_value < 0.05:
                            worksheet.cell(row=row, column=col).fill = red_fill

# Calculating Number of Interactions

In [ ]:
# red
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])['num_interactions'].values==0):
                    #     continue
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                if df_inetractions_timepoint.shape[0]==0:continue
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [ ]:
df_inetractions_all['type']='red'
df_num_interctions=df_inetractions_all

In [121]:
# syn
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])['num_interactions'].values==0):
                    #     print(group,epoch,timepoint,rat,order)
                    #     print((df_order))
                        
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [122]:
df_inetractions_all['type']='syn'
df_num_interctions=pd.concat([df_num_interctions,df_inetractions_all])

In [123]:
if condition=='4':
    t=['BI','4D','7D','10D']
    df_num_interctions=df_num_interctions[df_num_interctions.timepoint.isin(t)]

In [124]:
df_num_interctions['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                       ,inplace=True)

## Repeated measure

In [126]:
results=pd.DataFrame()
for _type in ['red','syn']:
    df_type=df_num_interctions[df_num_interctions['type']==_type]
    for group in [3,5]:
        df_group = df_type[(df_type.group==group)]
        for epoch in df_group.epoch.unique():
            df_epoch=df_group[df_group.epoch==epoch]
            for order in df_epoch.order.unique():
                df_order=df_epoch[df_epoch.order==order]
    #             min_timepoint =df_order.groupby('timepoint').count().min().min()
    #             df_order = keep_n_rows_per_category(df_order,'timepoint',min_timepoint)

                if df_order.shape[0]==0 or df_order.timepoint.unique().shape[0]==1 \
                or df_order.num_interactions.unique().shape[0]==1: continue


                p_value,esq =rm_test(df_order)
                row = pd.DataFrame([{'group':group,'epoch':epoch,'interactions order':order,'p_value':p_value,'type':_type,'Effect Size':esq}])
                results = pd.concat([results,row])
    results.reset_index(drop=True,inplace=True)

35.40477058215532 124 4
0.27003975485129433
29.304611870999725 124 4
0.2192050989249977
12.166504087889006 74 4
0.13095005839841437
7.20023767082597 74 4
0.06000339529751386
36.938359413675194 124 4
0.2828196617806266
24.61638738552409 124 4
0.18013656154603408
24.026031805526294 74 4
0.3003718829360899
6.2756916996047485 74 4
0.04679559570863926
34.75238155030732 124 4
0.26460317958589435
31.327573253193346 124 4
0.23606311044327788
11.020531400966084 74 4
0.1145790200138012
41.2270530821443 124 4
0.31855877568453583
2.757575757575142 124 4
-0.0020202020202071515
29.184119662388884 74 4
0.3740588523198412


In [127]:
# post hoc
for _type in results['type'].unique():
    df_type=results[results['type']==_type]
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_{_type}_{condition}_timepoints_cohen.xlsx',engine='openpyxl') as writer:
        for group in df_type.group.unique():
            gp_results=df_type[df_type.group==group]
            for epoch in gp_results.epoch.unique():
                epoch_resluts=gp_results[gp_results.epoch==epoch]
                df_results_all_orders=pd.DataFrame([])
                effect_sizes_all=pd.DataFrame([])
                for order in epoch_resluts['interactions order'].unique():
                    if epoch_resluts[epoch_resluts['interactions order']==order]['p_value'].values <0.05:
                        data=df_num_interctions[(df_num_interctions.group==group)&(df_num_interctions.epoch==epoch)&(df_num_interctions.order==order)&(df_num_interctions['type']==_type)]
                        df_posthoc = sp.posthoc_conover(data, val_col='num_interactions', group_col='timepoint', p_adjust = 'holm')
                        # df_posthoc=df_posthoc.astype('float128')

                        effect_sizes=calculate_effect_size(data,df_posthoc,'cohen',column='num_interactions')
                        # Convert to DataFrame
                        effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                        effect_sizes_df['order']=order
                        effect_sizes_df=effect_sizes_df.set_index(['order',effect_sizes_df.index])
                        effect_sizes_all=pd.concat([effect_sizes_all,effect_sizes_df])
                        
                        df_posthoc['order']=order
                        df_posthoc=df_posthoc.set_index(['order',df_posthoc.index])
                        df_results_all_orders=pd.concat([df_results_all_orders,df_posthoc])
                    
                    # add pvalue-kw
                    # new_row_data={0:epoch_resluts[epoch_resluts['interactions order']==order]['p_value'].values}
                    new_index = pd.MultiIndex.from_tuples([(order,'KW-pvalue'),(order,'Effect Size')], names=['order', None])
                    # new_row = pd.DataFrame(new_row_data, index=new_index)
                    new_row = pd.DataFrame([epoch_resluts[epoch_resluts['interactions order']==order]['p_value'].values
                                            ,epoch_resluts[epoch_resluts['interactions order']==order]['Effect Size'].values], index=new_index,columns=[0])
                    df_results_all_orders=pd.concat([new_row,df_results_all_orders])
                    df_results_all_orders=df_results_all_orders.sort_index()
                    

                
                
                df_results_all_orders.to_excel(writer, sheet_name=f'group {group} - epoch {epoch}')
                effect_sizes_all.to_excel(writer, sheet_name=f'ES - group {group} - epoch {epoch}')
                
                # formating values in exel file
                # Get the xlsxwriter workbook and worksheet objects
                workbook = writer.book
                worksheet = writer.sheets[f'group {group} - epoch {epoch}']
                
                # Define a cell format with red fill for cells greater than 5
                red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')

                # Iterate through the DataFrame and apply formatting to cells greater than 5
                for row in range(2, df_results_all_orders.shape[0] + 3):
                    for col in range(3, df_results_all_orders.shape[1] + 3):
                        cell_value = worksheet.cell(row=row, column=col).value
                        if cell_value is not None and cell_value != '' and cell_value < 0.05:
                            worksheet.cell(row=row, column=col).fill = red_fill

/tmp/ipykernel_204067/2697543048.py:12: RuntimeWarning: invalid value encountered in scalar divide
  return (u1 - u2) / s
/tmp/ipykernel_204067/2697543048.py:12: RuntimeWarning: invalid value encountered in scalar divide
  return (u1 - u2) / s
/tmp/ipykernel_204067/2697543048.py:12: RuntimeWarning: invalid value encountered in scalar divide
  return (u1 - u2) / s


In [93]:
# red
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])['num_interactions'].values==0):
                    #     continue
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                if df_inetractions_timepoint.shape[0]==0:continue
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [94]:
df_inetractions_all['type']='red'
df_num_interctions=df_inetractions_all

In [95]:
# syn
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])['num_interactions'].values==0):
                    #     print(group,epoch,timepoint,rat,order)
                    #     print((df_order))
                        
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [96]:
df_inetractions_all['type']='syn'
df_num_interctions=pd.concat([df_num_interctions,df_inetractions_all])

In [97]:
df_num_interctions['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                       ,inplace=True)

In [98]:
results=pd.DataFrame()
for _type in ['red','syn']:
    df_type=df_num_interctions[df_num_interctions['type']==_type]
    for group in [3,5]:
        df_group = df_type[(df_type.group==group)]
        for rat in df_group.rat.unique():
            df_rat = df_group[df_group.rat==rat]
            for epoch in df_rat.epoch.unique():
                df_epoch=df_rat[df_rat.epoch==epoch]
                for order in df_epoch.order.unique():
                    df_order=df_epoch[df_epoch.order==order]
        #             min_timepoint =df_order.groupby('timepoint').count().min().min()
        #             df_order = keep_n_rows_per_category(df_order,'timepoint',min_timepoint)
    
                    if df_order.shape[0]==0 or df_order.timepoint.unique().shape[0]==1 \
                    or df_order.num_interactions.unique().shape[0]==1: continue
    
    
                    p_value,esq =rm_test(df_order)
                    row = pd.DataFrame([{'group':group,'epoch':epoch,'interactions order':order,'rat':rat,'p_value':p_value,'type':_type,'Effect Size':esq}])
                    results = pd.concat([results,row])
        results.reset_index(drop=True,inplace=True)

35.40477058215532 124 4
0.27003975485129433
29.304611870999725 124 4
0.2192050989249977
12.166504087889006 74 4
0.13095005839841437
7.20023767082597 74 4
0.06000339529751386
2.000000000000001 6 2
0.2500000000000002
4.557801040269593 97 3
0.02721064936457014
2.0776054421769614 97 3
0.0008255898103932066
0.38502673796793463 28 2
-0.023652817770464053
0.09995886466475448 28 2
-0.03461696674366329
14.385299477630971 122 3
0.1040781468708485
70.9679725778408 122 3
0.5795627947717714
1.0079737699145783 72 2
0.0001139109987796903
0.09930069930066596 72 2
-0.012867132867133342
0.36111111111110983 14 2
-0.053240740740740845
0.36111111111110983 14 2
-0.053240740740740845
8.39399092970521 68 2
0.11203016560159408
4.4175824175824 68 2
0.05178155178155152
1.1017346460384347 48 2
0.0022116227399659725
0.44761904761902377 48 2
-0.012008281573499481
6.666666666666666 9 2
0.8095238095238094
3.199999999999995 9 2
0.31428571428571356
32.35902351304842 113 4
0.26934883956925154
10.063601992560619 113 4
0.

In [99]:
# post hoc
for rat in results.rat.unique():
    rat_results=results[results.rat==rat]
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_{rat}_{condition}_timepoints_cohen.xlsx',engine='openpyxl') as writer:
        for _type in rat_results['type'].unique():
            df_type=rat_results[rat_results['type']==_type]
            for group in df_type.group.unique():
                gp_results=df_type[df_type.group==group]
                for epoch in gp_results.epoch.unique():
                    epoch_resluts=gp_results[gp_results.epoch==epoch]
                    df_results_all_orders=pd.DataFrame([])
                    effect_sizes_all=pd.DataFrame([])
                    for order in epoch_resluts['interactions order'].unique():
                        if epoch_resluts[epoch_resluts['interactions order']==order]['p_value'].values <0.05:
                            data=df_num_interctions[(df_num_interctions.group==group)&(df_num_interctions.epoch==epoch)&(df_num_interctions.order==order)&(df_num_interctions['type']==_type)&(df_num_interctions['rat']==rat)]
                            df_posthoc = sp.posthoc_conover(data, val_col='num_interactions', group_col='timepoint', p_adjust = 'holm')

                            effect_sizes=calculate_effect_size(data,df_posthoc,'cohen',column='num_interactions')

                            # Convert to DataFrame
                            effect_sizes_df = pd.DataFrame(effect_sizes, columns=df_posthoc.columns, index=df_posthoc.index)
                            effect_sizes_df['order']=order
                            effect_sizes_df=effect_sizes_df.set_index(['order',effect_sizes_df.index])
                            effect_sizes_all=pd.concat([effect_sizes_all,effect_sizes_df])
                            
                            df_posthoc['order']=order
                            df_posthoc=df_posthoc.set_index(['order',df_posthoc.index])
                            df_results_all_orders=pd.concat([df_results_all_orders,df_posthoc])
                        
                        # add pvalue-kw
                        # new_row_data={0:epoch_resluts[epoch_resluts['interactions order']==order]['p_value'].values}
                        new_index = pd.MultiIndex.from_tuples([(order,'KW-pvalue'),(order,'Effect Size')], names=['order', None])
                        # new_row = pd.DataFrame(new_row_data, index=new_index)
                        new_row = pd.DataFrame([epoch_resluts[epoch_resluts['interactions order']==order]['p_value'].values
                                                ,epoch_resluts[epoch_resluts['interactions order']==order]['Effect Size'].values], index=new_index,columns=[0])
                        df_results_all_orders=pd.concat([new_row,df_results_all_orders])
                        df_results_all_orders=df_results_all_orders.sort_index()
                        
    
                    
                    if 0 not in df_posthoc.columns:
                        df_results_all_orders=df_results_all_orders.rename({0:'ES'},axis=1)
                    df_average_inetractions=a=df_num_interctions[(df_num_interctions.group==group)&(df_num_interctions.epoch==epoch)&(df_num_interctions['type']==_type)&(df_num_interctions['rat']==rat)]
                    df_average_inetractions=df_average_inetractions.groupby(['order','timepoint']).mean().drop(['group','session'],axis=1)
                    
                    df_average_inetractions.to_excel(writer, sheet_name=f'Average group {group} - epoch {epoch} - {_type}')
                    df_results_all_orders.to_excel(writer, sheet_name=f'group {group} - epoch {epoch} - {_type}')
                    effect_sizes_all.to_excel(writer, sheet_name=f'ES - group {group} - epoch {epoch} - {_type}')
                    
                    # formating values in exel file
                    # Get the xlsxwriter workbook and worksheet objects
                    workbook = writer.book
                    worksheet = writer.sheets[f'group {group} - epoch {epoch} - {_type}']
                    
                    # Define a cell format with red fill for cells greater than 5
                    red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')
    
                    # Iterate through the DataFrame and apply formatting to cells greater than 5
                    for row in range(2, df_results_all_orders.shape[0] + 3):
                        for col in range(3, df_results_all_orders.shape[1] + 3):
                            cell_value = worksheet.cell(row=row, column=col).value
                            if cell_value is not None and cell_value != '' and cell_value < 0.05:
                                worksheet.cell(row=row, column=col).fill = red_fill

/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applic

# Distance each rat from avg

In [100]:
# red
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])['num_interactions'].values==0):
                    #     continue
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                if df_inetractions_timepoint.shape[0]==0:continue
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [101]:
df_inetractions_all['type']='red'
df_num_interctions=df_inetractions_all

In [102]:
# syn
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])['num_interactions'].values==0):
                    #     print(group,epoch,timepoint,rat,order)
                    #     print((df_order))
                        
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [103]:
df_inetractions_all['type']='syn'
df_num_interctions=pd.concat([df_num_interctions,df_inetractions_all])

In [104]:
df_num_interctions['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                       ,inplace=True)

In [218]:
results=pd.DataFrame([])
for _type in ['red','syn']:
    df_type=df_num_interctions[df_num_interctions['type']==_type]
    for group in [3,5]:
        df_group = df_type[(df_type.group==group)]
        for epoch in df_group.epoch.unique():
            df_epoch=df_group[df_group.epoch==epoch]
            for timepoint in df_epoch.timepoint.unique():
                df_time=df_epoch[df_epoch.timepoint==timepoint]
                for order in df_time.order.unique():
                    df_order=df_time[df_time.order==order]
                    for rat in df_order.rat.unique():
                        df_rat=df_order[df_order.rat==rat]
                        df_other_rats=df_order[df_order.rat!=rat]
                        if len(np.unique(np.concatenate([df_rat['num_interactions'].values,df_other_rats['num_interactions'].values])))==1:
                            p_value=1
                        elif df_rat['num_interactions'].values.shape[0]==0 or df_other_rats['num_interactions'].values.shape[0]==0:
                            p_value='no data to compare'
                        else:
                            p_value=stats.kruskal(df_rat['num_interactions'].values,df_other_rats['num_interactions'].values)[1]
                        # print(rat)
                        row = pd.DataFrame([{'group':group,'epoch':epoch,'rat':rat,'timepoint':timepoint,'interactions order':order,'p_value':p_value,'type':_type}])
                        # print(row)
    
        #             min_timepoint =df_order.groupby('timepoint').count().min().min()
        #             df_order = keep_n_rows_per_category(df_order,'timepoint',min_timepoint)
    
                    # if df_order.shape[0]==0 or df_order.timepoint.unique().shape[0]==1 \
                    # or df_order.num_interactions.unique().shape[0]==1: continue
    
    
                    # p_value,esq =rm_test(df_order)
                    # row = pd.DataFrame([{'group':group,'epoch':epoch,'interactions order':order,'p_value':p_value,'type':_type,'Effect Size':esq}])
                    results = pd.concat([results,row])
results.reset_index(drop=True,inplace=True)

   group  epoch    rat  timepoint  interactions order   p_value type
0      3  SLEEP  PILO1          7                   3  0.299387  red
   group  epoch    rat  timepoint  interactions order   p_value type
0      3  SLEEP  PILO2          7                   3  0.376159  red
   group  epoch   rat  timepoint  interactions order   p_value type
0      3  SLEEP  R322          7                   3  0.000183  red
   group  epoch   rat  timepoint  interactions order   p_value type
0      3  SLEEP  R358          7                   3  0.376159  red
   group  epoch    rat  timepoint  interactions order  p_value type
0      3  SLEEP  PILO1          7                   4        1  red
   group  epoch    rat  timepoint  interactions order  p_value type
0      3  SLEEP  PILO2          7                   4        1  red
   group  epoch   rat  timepoint  interactions order  p_value type
0      3  SLEEP  R322          7                   4        1  red
   group  epoch   rat  timepoint  interactions

In [256]:
with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'stat_distance_from_avg.xlsx',engine='openpyxl') as writer:
    for group in [3,5]:
        df_group = df_num_interctions[(df_num_interctions.group==group)] 
        for rat in df_group.rat.unique():
            df_rat=df_group[df_group.rat==rat]
            df_other_rats=df_group[df_group.rat!=rat]
            
            for epoch in df_rat.epoch.unique():
                df_epoch=df_rat[df_rat.epoch==epoch]
                df_epoch_other=df_other_rats[df_other_rats.epoch==epoch]
                
                for _type in ['red','syn']:
                    df_type=df_epoch[df_epoch.type==_type]
                    df_type_other=df_epoch_other[df_epoch_other.type==_type]
                    
                    results = pd.DataFrame([])
                    for timepoint in df_type.timepoint.unique():
                        df_time=df_type[df_type.timepoint==timepoint]
                        df_time_other=df_type_other[df_type_other.timepoint==timepoint]

                        for order in df_time.order.unique():
                            df_order=df_time[df_time.order==order]
                            df_order_other=df_time_other[df_time_other.order==order]
                        
                            #stats
                            if len(np.unique(np.concatenate([df_order['num_interactions'].values,df_order_other['num_interactions'].values])))==1:
                                p_value=1
                            elif df_order['num_interactions'].values.shape[0]==0 or df_order_other['num_interactions'].values.shape[0]==0:
                                p_value='no data to compare'
                            else:
                                p_value=stats.kruskal(df_order['num_interactions'].values,df_order_other['num_interactions'].values)[1]
                        
                            row = pd.DataFrame([{'timepoint':timepoint,'interactions order':order,'p_value':p_value}])
                            print(row)
                            results = pd.concat([results,row])

                        # results.reset_index(drop=True,inplace=True)
                        # results.set_index(['interactions order','timepoint'],inplace=True)
                    results=results.set_index(['timepoint','interactions order'])
                    # results=results.sort_index()
                    results.to_excel(writer, sheet_name=f'stat {rat} - group {group} - epoch {epoch} - {_type}')

                    #formating values in exel file
                   # Get the xlsxwriter workbook and worksheet objects
                    workbook = writer.book
                    worksheet = writer.sheets[f'stat {rat} - group {group} - epoch {epoch} - {_type}']
                    
                     # Iterate through the DataFrame and apply formatting to cells greater than 5
                    for row in range(2, results.shape[0] + 3):
                        for col in range(3, results.shape[1] + 3):
                            cell_value = worksheet.cell(row=row, column=col).value
                            if cell_value == 'no data to compare':
                                continue
                            elif cell_value is not None and cell_value != '' and cell_value < 0.05:
                                worksheet.cell(row=row, column=col).fill = red_fill
                    

   timepoint  interactions order   p_value
0          7                   3  0.299387
   timepoint  interactions order  p_value
0          7                   4        1
   timepoint  interactions order   p_value
0          7                   3  0.299387
   timepoint  interactions order  p_value
0          7                   4        1
   timepoint  interactions order   p_value
0         10                   3  0.609575
   timepoint  interactions order   p_value
0         10                   4  0.146238
   timepoint  interactions order   p_value
0         14                   3  0.377538
   timepoint  interactions order   p_value
0         14                   4  0.227195
   timepoint  interactions order  p_value
0         25                   3  0.00599
   timepoint  interactions order   p_value
0         25                   4  0.000406
   timepoint  interactions order       p_value
0          7                   3  5.292013e-13
   timepoint  interactions order       p_value
0    

/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applic

   timepoint  interactions order   p_value
0         10                   3  0.006684
   timepoint  interactions order   p_value
0         10                   4  0.000576
   timepoint  interactions order  p_value
0         14                   3  0.12663
   timepoint  interactions order  p_value
0         14                   4        1
   timepoint  interactions order   p_value
0         25                   3  0.836503
   timepoint  interactions order   p_value
0         25                   4  0.345231
   timepoint  interactions order   p_value
0          7                   3  0.136781
   timepoint  interactions order   p_value
0          7                   4  0.106357
   timepoint  interactions order   p_value
0         10                   3  0.000051
   timepoint  interactions order   p_value
0         10                   4  0.044688
   timepoint  interactions order  p_value
0         14                   3        1
   timepoint  interactions order  p_value
0         14      

/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applic

   timepoint  interactions order   p_value
0         14                   3  0.199543
   timepoint  interactions order  p_value
0         14                   4        1
   timepoint  interactions order  p_value
0          7                   3  0.00002
   timepoint  interactions order  p_value
0          7                   4        1
   timepoint  interactions order   p_value
0         10                   3  0.041902
   timepoint  interactions order   p_value
0         10                   4  0.426464
   timepoint  interactions order   p_value
0          7                   3  0.002278
   timepoint  interactions order   p_value
0          7                   4  0.018636
   timepoint  interactions order   p_value
0         10                   3  0.052378
   timepoint  interactions order   p_value
0         10                   4  0.336525
   timepoint  interactions order   p_value
0          7                   3  0.004096
   timepoint  interactions order  p_value
0          7      

...

/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applic

   timepoint  interactions order   p_value
0          4                   4  0.302796
   timepoint  interactions order  p_value
0          0                   3  0.01717
   timepoint  interactions order       p_value
0          0                   4  2.514045e-08
   timepoint  interactions order   p_value
0         10                   3  0.001454
   timepoint  interactions order   p_value
0         10                   4  0.466403
   timepoint  interactions order   p_value
0         14                   3  0.051028
   timepoint  interactions order   p_value
0         14                   4  0.003175
   timepoint  interactions order   p_value
0         25                   3  0.243887
   timepoint  interactions order   p_value
0         25                   4  0.268925
   timepoint  interactions order   p_value
0          4                   3  0.389927
   timepoint  interactions order   p_value
0          4                   4  0.105193
   timepoint  interactions order   p_value
0    

/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applic

   timepoint  interactions order  p_value
0          0                   3  0.07646
   timepoint  interactions order  p_value
0          0                   4  0.66375
   timepoint  interactions order   p_value
0         10                   3  0.358949
   timepoint  interactions order   p_value
0         10                   4  0.443946
   timepoint  interactions order  p_value
0          4                   3  0.06245
   timepoint  interactions order   p_value
0          4                   4  0.350317
   timepoint  interactions order       p_value
0          7                   3  1.911653e-10
   timepoint  interactions order   p_value
0          7                   4  0.000068
   timepoint  interactions order   p_value
0          0                   3  0.588778
   timepoint  interactions order       p_value
0          0                   4  4.745456e-10
   timepoint  interactions order   p_value
0         10                   3  0.025787
   timepoint  interactions order   p_value
0

/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
/home/morteza/anaconda3/lib/python3.10/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


# Stats Interactions

In [163]:
def analysis_num_interactions_group(df,_type,group_number,region=False,SE_rats=True):
    if region:
        axis=None
    else:
        axis=1
    df_group = df[df.group==group_number]
    df_analysis_group=pd.DataFrame([])
    
    for rat in df_group.rat.unique():
        df_rat=df_group[df_group.rat==rat]
        
        for epoch in df_rat.epoch.unique():
            df_epoch=df_rat[df_rat.epoch==epoch]
            
            for timepoint in df_epoch.timepoint.unique():
                    df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                    
                    if SE_rats:
                        # remove sham rats
                        df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                    else:
                        # remove SE rats
                        df_timepoint=remove_SE_rats(df_timepoint,timepoint)
                    
                    
                    df_all_interactions=pd.DataFrame([])
                    for order in [3,4,5]:
                        if region:
                            unique_interactions=list(groups_ROIs[group_number].values())
                        else:
                            unique_interactions = find_unique_interactions(df_timepoint[f'index_var_{_type}_{order}'],axis)
                            if unique_interactions is None:
                                continue

                        # find interactions
                        df_single_order=get_inetreaction(df_timepoint[f'index_var_{_type}_{order}'],rat,epoch,timepoint,unique_interactions,region)
                        df_single_order['#session']=df_timepoint.shape[0]
                            
                        if region:
                            if df_all_interactions.shape[0]==0:
                                df_all_interactions=df_single_order
                            else:
                                diffrent_cols=df_single_order.columns.difference(df_all_interactions.columns)
                                if bool((diffrent_cols).any()):
                                    df_all_interactions[diffrent_cols]=0
                                    
                                df_all_interactions.loc[:, unique_interactions]=\
                        df_all_interactions[unique_interactions]+df_single_order.loc[:, unique_interactions]
                        else:
                            # get common columns
                            common_cols = list(set(df_all_interactions.columns).intersection(df_single_order.columns))
                            # get unique columns for each dataframe
                            df1_cols = [col for col in df_all_interactions.columns if col not in common_cols]
                            df2_cols = [col for col in df_single_order.columns if col not in common_cols]

                            if df_single_order[df2_cols].shape[1]==0:
                                continue
                            df_all_interactions=pd.concat([df_all_interactions.reset_index(drop=True)
                                                           , df_single_order[df2_cols].reset_index(drop=True)], axis=1)
                    if df_all_interactions.shape[0]==0:
                        df_all_interactions=pd.DataFrame([{'rat':rat,'epoch':epoch,'timepoint':timepoint}])

                        session_num=[n_session for n_session in range(df_timepoint.shape[0])]  
                        df_all_interactions['session']=[session_num]
                        df_all_interactions=df_all_interactions.explode('session',ignore_index=True)
                        df_all_interactions['#session']=df_timepoint.shape[0]
                            
                    df_analysis_group=pd.concat([df_analysis_group,df_all_interactions],ignore_index=True)
        df_analysis_group.reset_index(drop=True,inplace=True)
    return df_analysis_group


In [164]:
def find_unique_interactions(column,axis=0):
    data=column.dropna().values
    if data.shape[0]==0:
        return None
    data=np.vstack(data)
    unique = np.unique(data,axis=axis)
    return np.sort(unique)

In [165]:
def get_inetreaction(column,rat,epoch,timepoint,unique,region=False):
    df_interactions=pd.DataFrame([])
    data=column.values
    index_nans=np.where(pd.isna(data))[0]
#     data=np.vstack(data)
    for session in range(data.shape[0]):
        df_session=pd.DataFrame([{'rat':rat,'epoch':epoch,'timepoint':timepoint}])
        if region:
#             print(unique)
            for ineteraction in unique:
                if session in index_nans:
                    count=0
                else:
                    count=np.count_nonzero(data[session] == ineteraction)
                df_session[''.join(ineteraction)]=count
                df_session['session']=session
            df_interactions=pd.concat([df_interactions,df_session])
        else:
            for ineteraction in unique:
                if session in index_nans:
                    count=0
                else:
                    count=np.count_nonzero(np.all(np.sort(data[session]) == np.sort(ineteraction),axis=1))
                df_session[','.join(ineteraction)]=count
                df_session['session']=session
            df_interactions=pd.concat([df_interactions,df_session], axis=0, ignore_index=True)
    return df_interactions

In [189]:
def stat_inteactions(df_stat,epoch):
    df_stat=df_stat[df_stat.epoch==epoch]
    pairs=[]
    p_vals=[]
    effect_sizes_df=pd.DataFrame([])
    for interaction in df_stat.iloc[:,3:].columns:
        if interaction in ['epoch','#session','rat','timepoint','session']:continue
        df_interaction = df_stat[['timepoint',interaction] ]   
        if len(df_interaction.timepoint.unique()==1):continue
        p_rm,esq=rm_test(df_interaction,interaction)
        # print('y',esq)
        p_vals.append(p_rm)
        pairs.append(((interaction,'KW-pvalue'),(interaction,''),(interaction,esq)))
        # p_vals.append(esq)
        # pairs.append(((interaction,'Effect Size'),(interaction,''),(interaction,'')))
        if p_rm>=0.05:continue
        # post hoc
        p_post_hoc=sp.posthoc_conover(df_interaction, val_col=interaction, group_col='timepoint', p_adjust = 'holm')

       
        # try:
        print(epoch)
        print(interaction)
        effect_sizes_=calculate_effect_size(df_interaction,p_post_hoc,'cohen',column=interaction)
        # print(effect_sizes_.shape)
        # print(p_post_hoc.shape)

        # except:
        #     print(epoch,interaction)
        #     print(df_stat[[interaction,'timepoint']])
        #     print(p_post_hoc.shape)
        
        # Convert to DataFrame
        effect_sizes_df = pd.DataFrame(effect_sizes_, columns=p_post_hoc.columns, index=p_post_hoc.index)
        
        
        for col in p_post_hoc.columns:
            for i in p_post_hoc.index:
                p_value=p_post_hoc.loc[i,col] 
                p_vals.append(p_value)
                pairs.append(((interaction,i),(interaction,col),(interaction,effect_sizes_df.loc[i,col])))
    return pairs,p_vals,effect_sizes_df

In [190]:
# def rm_test(df_test,col):
# #     if pg.normality(df_test.num_interactions)['normal'].values[0]:
# #         result = pg.rm_anova(df_test,dv='num_interactions',within='timepoint',subject='order')
# #         print('yes')
# #     else:
#     pval = stats.kruskal(*list(df_test.groupby('timepoint').agg(list)[f'{col}'].values))[1]
#     return pval

In [191]:
def conver_pair_pvalue(pairs_interactions,p_values):
    df_pvalue=pd.DataFrame(pairs_interactions)
    df_pvalue = df_pvalue.apply(lambda x: [x[0][0],x[0][1],x[1][1],x[2][1]], axis=1)
    df_pvalue=pd.DataFrame(df_pvalue)
    df_pvalue[['interactions', 'time1', 'time2','effect size']] = pd.DataFrame(df_pvalue[0].tolist(), index=df_pvalue.index)
    df_pvalue = df_pvalue.drop(0, axis=1)
    df_pvalue=pd.concat([df_pvalue,pd.DataFrame(p_values,columns=['p_value'])],axis=1)
    # sorting
    timepoints=sort_date(order=all_timepoints,timepoints=df_summary_ineractions.timepoint.unique().tolist())
    timepoints=['KW-pvalue','Effect Size'] + timepoints

    df_pvalue['time1'] = pd.Categorical(
    df_pvalue['time1'], 
    categories=timepoints, 
    ordered=True)

    df_pvalue['time2'] = pd.Categorical(
    df_pvalue['time2'], 
    categories=timepoints, 
    ordered=True)

    df_pvalue=df_pvalue.sort_values(['interactions','time1','time2'])
    df_pvalue=df_pvalue[['interactions', 'time1', 'time2','p_value','effect size']]
    return df_pvalue

In [200]:
#stat interactions
for _type in ['red','syn']:
    for group_num in [3,5]:
            df_summary_ineractions=analysis_num_interactions_group(df,_type,group_num).fillna(0)

            df_summary_stat=df_summary_ineractions

            for epoch in df_summary_ineractions.epoch.unique():
                pairs,pvalues,effect_sizes_df=stat_inteactions(df_summary_stat,epoch)
                if (effect_sizes).shape[0]==0 or len(pvalues)==0:continue
                with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'stats_inteactions_{_type}_cohen.xlsx',engine='openpyxl') as writer:
                    df_pvalues = conver_pair_pvalue(pairs,pvalues)
                    #saving
                    df_pvalues.to_excel(writer, sheet_name=f'group {group_num} - epoch {epoch}',index=False)
                    # effect_sizes_df.to_excel(writer, sheet_name=f'ES - group {group_num} - epoch {epoch}')

                    # formating values in exel file
                    # Get the xlsxwriter workbook and worksheet objects
                    workbook = writer.book
                    worksheet = writer.sheets[f'group {group_num} - epoch {epoch}']

                    # Define a cell format with red fill for cells greater than 5
                    red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')

                    # Iterate through the DataFrame and apply formatting to cells greater than 5
                    for row in range(2, df_pvalues.shape[0] + 73):
                        for col in range(4, df_pvalues.shape[1]):
                            cell_value = worksheet.cell(row=row, column=col).value
                            if cell_value is not None and cell_value != '' and cell_value < 0.05:
                                worksheet.cell(row=row, column=col).fill = red_fill

In [202]:
#stat interactions
for _type in ['red','syn']:

    for group_num in [3,5]:
            df_summary_ineractions=analysis_num_interactions_group(df,_type,group_num,True).fillna(0)

            df_summary_stat=df_summary_ineractions

            for epoch in df_summary_ineractions.epoch.unique():
                pairs,pvalues,effect_sizes_df=stat_inteactions(df_summary_stat,epoch)
                if (effect_sizes).shape[0]==0 or len(pvalues)==0:continue
                with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'stats_region_{_type}.xlsx',engine='openpyxl') as writer:
                    df_pvalues = conver_pair_pvalue(pairs,pvalues)
                    df_pvalues=df_pvalues.rename({'interactions':'region'},axis=1)
                    #saving
                    df_pvalues.to_excel(writer, sheet_name=f'group {group_num} - epoch {epoch}',index=False)
                    # effect_sizes_df.to_excel(writer, sheet_name=f'ES - group {group_num} - epoch {epoch}')

                    # formating values in exel file
                    # Get the xlsxwriter workbook and worksheet objects
                    workbook = writer.book
                    worksheet = writer.sheets[f'group {group_num} - epoch {epoch}']

                    # Define a cell format with red fill for cells greater than 5
                    red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')

                    # Iterate through the DataFrame and apply formatting to cells greater than 5
                    for row in range(2, df_pvalues.shape[0] + 73):
                        for col in range(4, df_pvalues.shape[1] + 3):
                            cell_value = worksheet.cell(row=row, column=col).value
                            if cell_value is not None and type(cell_value) != str and cell_value < 0.05:
                                worksheet.cell(row=row, column=col).fill = red_fill

# Analysis of Number of interactions

In [203]:
# red
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_red_{order}'])['num_interactions'].values==0):
                    #     continue
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                if df_inetractions_timepoint.shape[0]==0:continue
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [204]:
df_inetractions_all['type']='red'
df_num_interctions=df_inetractions_all

In [205]:
# syn
df_inetractions_all=pd.DataFrame()
for group in [3,5]:
    df_group=df[df.group==group]
    df_inetractions_group=pd.DataFrame()
    for rat in df_group.rat.unique().tolist():
        df_rat=df_group[df_group.rat==rat]
        df_inetractions_rat=pd.DataFrame()
        for epoch in df_rat.epoch.unique().tolist():
            df_epoch=df_rat[df_rat.epoch==epoch]
            df_inetractions_epoch=pd.DataFrame()
            for timepoint in df_epoch.timepoint.unique().tolist():
                df_inetractions_timepoint=pd.DataFrame()
                df_timepoint=df_epoch[df_epoch.timepoint==timepoint]
                # remove SE rats for non BI timepoints
                df_timepoint=remove_sham_rats(df_timepoint,timepoint)
                df_inetractions_timepoint=pd.DataFrame()
                for order in [3,4]:
                    df_order=get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])
                    # if all(get_interactions_per_session(df_timepoint[f'index_var_syn_{order}'])['num_interactions'].values==0):
                    #     print(group,epoch,timepoint,rat,order)
                    #     print((df_order))
                        
                    df_order['order']=order
                    df_inetractions_timepoint=pd.concat([df_inetractions_timepoint,df_order])
                df_inetractions_timepoint['timepoint']=timepoint
                df_inetractions_epoch=pd.concat([df_inetractions_epoch,df_inetractions_timepoint])
            df_inetractions_epoch['epoch']=epoch
            df_inetractions_rat=pd.concat([df_inetractions_rat,df_inetractions_epoch])
        df_inetractions_rat['rat']=rat
        df_inetractions_group=pd.concat([df_inetractions_group,df_inetractions_rat])
    df_inetractions_group['group']=group    
    df_inetractions_all=pd.concat([df_inetractions_all,df_inetractions_group])
df_inetractions_all.num_interactions=df_inetractions_all.num_interactions.astype(float)

In [206]:
df_inetractions_all['type']='syn'
df_num_interctions=pd.concat([df_num_interctions,df_inetractions_all])

In [207]:
df_red_syn_interactions = df_num_interctions.groupby(['group','epoch','timepoint','order','type'])[['num_interactions']].sum()
df_red_syn_interactions.reset_index(inplace=True)
df_red_syn_interactions = pd.pivot(df_red_syn_interactions,index=['group', 'epoch', 'timepoint'],columns=['type','order'],values='num_interactions').fillna(0)


# calculate TOTAL row
df_count=remove_sham_rats(df,'ALLTIMEPOINTS')
df_count.groupby(['group','epoch','timepoint']).count()
# df_count['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                       # ,inplace=True)

df_red_syn_interactions['#epochs'] = df_count.groupby(['group','epoch','timepoint']).count()['num_syn_total']
####

#devid by number of epochos
df_red_syn_interactions.iloc[:,:-1] = df_red_syn_interactions.iloc[:,:-1].div(df_red_syn_interactions['#epochs'], axis=0)

df_total=df_red_syn_interactions.groupby(level=[0,1]).sum(numeric_only=True).assign(Number='Total')
df_total=df_total.set_index([df_total.index,'Number'])
# Add row totals to the end of each 2-level index
df_red_syn_interactions = pd.concat([df_red_syn_interactions, df_total])
# sorting based on index
df_red_syn_interactions=df_red_syn_interactions.sort_index().apply(lambda x: round(x, 3))

# #save
df_red_syn_interactions.to_excel(SAVE_PATH_ANALYSIS/'num_red_syn_interactions.xlsx')
df_red_syn_interactions

type                      red    syn    red    syn #epochs
order                       3      3      4      4        
group epoch timepoint                                     
3     SLEEP 7D          4.000  0.000  1.000  0.000       5
            Total       4.000  0.000  1.000  0.000       5
      SN    10D         2.901  0.268  0.746  0.000      71
            14D         3.333  0.111  0.778  0.000       9
            25D         0.727  1.545  0.000  0.273      11
            7D          1.485  1.152  0.394  0.000      33
            Total       8.447  3.076  1.918  0.273     124
      VC    10D         3.625  0.000  1.000  0.000      32
            14D         4.000  0.000  1.000  0.000       7
            25D         2.727  0.000  0.818  0.000      11
            7D          3.083  0.208  0.833  0.000      24
            Total      13.436  0.208  3.652  0.000      74
5     SLEEP 7D          4.000  0.000  1.000  0.000       5
            Total       4.000  0.000  1.000  0.000       5
      SN    10D         3.000  0.268  0.817  0.000      71
            14D         4.000  0.000  1.000  0.000       9
            25D         1.455  1.636  0.182  0.000      11
            7D          1.848  1.091  0.788  0.030      33
            Total      10.303  2.995  2.787  0.030     124
      VC    10D         3.781  0.000  1.000  0.000      32
            14D         4.000  0.000  1.000  0.000       7
            25D         2.727  0.818  0.818  0.000      11
            7D          3.542  0.333  0.875  0.000      24
            Total      14.050  1.152  3.693  0.000      74

In [210]:
def ovalues_rm_test(df_test,col):
    pval = stats.kruskal(*list(df_test.groupby('timepoint').agg(list)[f'{col}'].values))[1]
    return pval

In [212]:

def preprocess_o_values(df,order,_type):
    df_analysis = df.explode([f'index_var_{_type}_{order}',f'sorted_{_type}_{order}']).reset_index(drop=True)
    df_analysis=df_analysis[df_analysis.group==group]
    df_analysis=df_analysis=remove_sham_rats(df_analysis,'ALL TIMEPOINTS')
    # remove nan rows
    df_analysis=df_analysis[~df_analysis[f'index_var_{_type}_{order}'].isna()]
    
    
    df_analysis=df_analysis.explode([f'sorted_{_type}_{order}']).reset_index(drop=True)
    return df_analysis


In [ ]:
results=pd.DataFrame()
for _type in ['red','syn']:
    with pd.ExcelWriter(SAVE_PATH_ANALYSIS/f'post_hoc_{_type}_ovalues.xlsx',engine='openpyxl') as writer:
        for group in [3,5]:
            df_group = df[(df.group==group)]
            for epoch in df_group.epoch.unique():
                df_epoch=df_group[df_group.epoch==epoch]
    
                df_results_all_orders=pd.DataFrame([])
                for order in [3,4]:
                    df_analysis=preprocess_o_values(df_epoch,order,_type)
                    data=np.hstack(df_analysis[f'sorted_{_type}_{order}'].dropna().values)
        #             min_timepoint =df_order.groupby('timepoint').count().min().min()
        #             df_order = keep_n_rows_per_category(df_order,'timepoint',min_timepoint)
                    
                    if data.shape[0]==0 or df_analysis.timepoint.unique().shape[0]==1: continue
    

                    p_value =ovalues_rm_test(df_analysis,f'sorted_{_type}_{order}')
                    df_analysis['timepoint'].replace({'BI':0,'4D':4, '7D':7, '10D':10, '12D':12, '14D':14, '15D':15, '16D':16, '25D':25}
                                       ,inplace=True)
                    if p_value<0.05:
                        df_posthoc = sp.posthoc_conover(df_analysis, val_col=f'sorted_{_type}_{order}', group_col='timepoint', p_adjust = 'holm')
                        df_posthoc['order']=order
                        df_posthoc=df_posthoc.set_index(['order',df_posthoc.index])
                        df_results_all_orders=pd.concat([df_results_all_orders,df_posthoc])
    
                    # add pvalue-kw
                    new_row_data={0:p_value}
                    new_index = pd.MultiIndex.from_tuples([(order,'KW-pvalue')], names=['order', None])
                    new_row = pd.DataFrame(new_row_data, index=new_index)
                    df_results_all_orders=pd.concat([new_row,df_results_all_orders])
                    df_results_all_orders=df_results_all_orders.sort_index()
    
                print(epoch)
                df_results_all_orders.to_excel(writer, sheet_name=f'group {group} - epoch {epoch}')

                # formating values in exel file
                # Get the xlsxwriter workbook and worksheet objects
                workbook = writer.book
                worksheet = writer.sheets[f'group {group} - epoch {epoch}']
                # Define a cell format with red fill for cells greater than 5
                red_fill = PatternFill(start_color='e66b63', end_color='e66b63', fill_type='solid')
    
                # Iterate through the DataFrame and apply formatting to cells greater than 5
                for row in range(2, df_results_all_orders.shape[0] + 73):
                    for col in range(4, df_results_all_orders.shape[1] + 3):
                        cell_value = worksheet.cell(row=row, column=col).value
                        if cell_value is not None and cell_value != '' and cell_value < 0.05:
                            worksheet.cell(row=row, column=col).fill = red_fill

# prediction